In [44]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

train_df = pd.read_csv(f'{INPUT_DIR}adult.data', header = None, names = COL_NAMES)
test_df = pd.read_csv(f'{INPUT_DIR}adult.test', header = None, names = COL_NAMES)

In [4]:
# train_raw_report = ProfileReport(test_df, explorative = True) 
# train_raw_report.to_file("train_raw_report.html")
# test_raw_report = ProfileReport(test_df, explorative = True) 
# test_raw_report.to_file("test_raw_report.html")

In [6]:
concat_df = pd.concat([train_df,test_df])
concat_df.drop(['education','capital-gain','capital-loss'],1, inplace = True)
concat_df = concat_df.replace([' ?'],value = np.nan).dropna()

concat_df = pd.get_dummies(concat_df, drop_first = False, columns = CAT_COL)
concat_df['is_test'] = concat_df['50k'].apply(lambda x : 1 if x in [' <=50K.', ' >50K.'] else 0)
concat_df['50k'] = concat_df['50k'].apply(lambda x: ' >50K' if x in [' >50K', ' >50K.'] else ' <=50K')

concat_df = pd.get_dummies(concat_df, drop_first = True, columns = ['50k'])

In [7]:
concat_df.reset_index(drop = True, inplace = True)

In [8]:
concat_copy = concat_df.copy()

In [22]:
standardize_list = list(concat_copy.columns)
standardize_list.remove('is_test')
standardize_list.remove('50k_ >50K')

In [23]:
from sklearn.preprocessing import StandardScaler

autoscaler = StandardScaler()
concat_copy[standardize_list] = autoscaler.fit_transform(concat_copy[standardize_list])

In [24]:
concat_copy

,age,fnlwgt,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,...,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,is_test,50k_ >50K
0,0.034201,-1.062295,-0.078120,-0.179133,-0.271285,-1.671940,-0.194353,-0.302710,4.715765,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,0.308506,-0.042881,-0.022558,0,0
1,0.866417,-1.007438,-2.326738,-0.179133,-0.271285,-1.671940,-0.194353,3.303493,-0.212055,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,0.308506,-0.042881,-0.022558,0,0
2,-0.041455,0.245284,-0.078120,-0.179133,-0.271285,0.598108,-0.194353,-0.302710,-0.212055,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,0.308506,-0.042881,-0.022558,0,0
3,1.093385,0.425853,-0.078120,-0.179133,-0.271285,0.598108,-0.194353,-0.302710,-0.212055,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,0.308506,-0.042881,-0.022558,0,0
4,-0.798015,1.407393,-0.078120,-0.179133,-0.271285,0.598108,-0.194353,-0.302710,-0.212055,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,-3.241430,-0.042881,-0.022558,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,-0.419735,0.525154,-0.078120,-0.179133,-0.271285,0.598108,-0.194353,-0.302710,-0.212055,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,0.308506,-0.042881,-0.022558,1,0
45218,0.034201,0.243135,-0.411249,-0.179133,-0.271285,0.598108,-0.194353,-0.302710,-0.212055,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,0.308506,-0.042881,-0.022558,1,0
45219,-0.041455,1.753613,0.754701,-0.179133,-0.271285,0.598108,-0.194353,-0.302710,-0.212055,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,0.308506,-0.042881,-0.022558,1,0
45220,0.412481,-1.001947,-0.078120,-0.179133,-0.271285,0.598108,-0.194353,-0.302710,-0.212055,-0.021554,...,-0.021035,-0.047312,-0.034896,-0.025332,-0.023985,0.308506,-0.042881,-0.022558,1,0


In [26]:
TARGET_COL

'50k_ >50K'

In [27]:
TEST_IND = 30162

In [28]:
concat_copy.drop('is_test',1, inplace = True)

In [29]:
X_test = concat_copy[TEST_IND:].drop(TARGET_COL,1)
y_test = concat_copy[TEST_IND:][TARGET_COL]

In [30]:
X_train = concat_copy[:TEST_IND].drop(TARGET_COL,1)
y_train = concat_copy[:TEST_IND][TARGET_COL]

In [31]:
from sklearn.model_selection import StratifiedKFold

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
X = np.array(X_train)

In [34]:
y = np.array(y_train)

In [35]:
from sklearn.svm import SVC

In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [43]:
skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
fold_no = 0
for train_index, test_index in skf.split(X, y):
    X_samp_train, X_samp_test = X[train_index], X[test_index]
    y_samp_train, y_samp_test = y[train_index], y[test_index]
    
    fold_no += 1
    print(f"Fold : {fold_no}          Model used : Logistic Regression")
    
    lc = LogisticRegression(max_iter = 20000, class_weight = "balanced").fit(X_samp_train,y_samp_train)
    
    print("Precision : {}".format(precision_score(y_samp_test, lc.predict(X_samp_test))))
    print("Recall : {}".format(recall_score(y_samp_test, lc.predict(X_samp_test))))
    print("f1_score : {}".format(f1_score(y_samp_test, lc.predict(X_samp_test))))
    array_ = confusion_matrix(y_samp_test, lc.predict(X_samp_test))
    print(array_)
#     df_cm = pd.DataFrame(array_, index = [i for i in ['1','0']], columns = [i for i in ['1','0']])
#     plt.figure(figsize = (10,7))
    #sn.heatmap(df_cm, annot=True)
    #plt.show()
    print("")
    print("")

Fold : 1          Model used : Logistic Regression
Precision : 0.5536347517730497
Recall : 0.8315579227696405
f1_score : 0.6647152740819585
[[3524 1007]
 [ 253 1249]]


Fold : 2          Model used : Logistic Regression
Precision : 0.5412414965986394
Recall : 0.8475366178428761
f1_score : 0.660612350804359
[[3452 1079]
 [ 229 1273]]


Fold : 3          Model used : Logistic Regression
Precision : 0.5527681660899654
Recall : 0.8514323784143905
f1_score : 0.6703383162863887
[[3497 1034]
 [ 223 1278]]


Fold : 4          Model used : Logistic Regression
Precision : 0.5510647544545849
Recall : 0.8447701532311792
f1_score : 0.667017359284587
[[3498 1033]
 [ 233 1268]]


Fold : 5          Model used : Logistic Regression
Precision : 0.5443812855268911
Recall : 0.8288948069241012
f1_score : 0.6571654790182107
[[3488 1042]
 [ 257 1245]]




In [ ]:
Fold : 1          Model used : Logistic Regression
Precision : 0.546284224250326
Recall : 0.8368841544607191
f1_score : 0.6610570602156193
[[3487 1044]
 [ 245 1257]]


Fold : 2          Model used : Logistic Regression
Precision : 0.556099732858415
Recall : 0.8315579227696405
f1_score : 0.6664887940234792
[[3534  997]
 [ 253 1249]]


Fold : 3          Model used : Logistic Regression
Precision : 0.5513816925734024
Recall : 0.8507661558960693
f1_score : 0.6691118679591301
[[3492 1039]
 [ 224 1277]]


Fold : 4          Model used : Logistic Regression
Precision : 0.5434969853574505
Recall : 0.8407728181212525
f1_score : 0.6602144912372483
[[3471 1060]
 [ 239 1262]]


Fold : 5          Model used : Logistic Regression
Precision : 0.5473233404710921
Recall : 0.8508655126498003
f1_score : 0.6661454261141516
[[3473 1057]
 [ 224 1278]]

In [197]:
import matplotlib.pyplot as plt
import seaborn as sn